In [14]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install -y chromium-chromedriver
!pip install xlsxwriter
!pip install --upgrade selenium


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (125 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0

In [15]:
from selenium import webdriver
from IPython.display import display,clear_output
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import HTML, display
from urllib.parse import urlparse
import pandas as pd
import xlsxwriter
import time

In [16]:
!pip install --upgrade chromedriver-autoinstaller


In [17]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--enable-logging")
options.add_argument("--log-level=0")



In [18]:
df=pd.DataFrame()
df_categories=pd.DataFrame()
df_Memmber=pd.DataFrame()
df_Social=pd.DataFrame()

driver = webdriver.Chrome(options=options)

df

""


In [19]:
def get_text(selector,attr=' '):
  global driver
  try:
    if attr==' ':
      return driver.find_element(By.CSS_SELECTOR,selector).text
    else:
      return driver.find_element(By.CSS_SELECTOR,selector).get_attribute(attr)
  except Exception as e:
    return ' '

In [20]:
def extract_social_media_links(social_media_links, company_index):
  global df,df_categories,df_Memmber,df_Social
  try:
      for link in social_media_links:
          parsed_url = urlparse(link.get_attribute('href'))
          hostname = parsed_url.hostname
          if hostname.startswith('www.'):
              hostname = hostname[4:]
          member_index = len(df_Memmber)
          df_Social.loc[member_index,'ID_Company'] = company_index
          df_Social.loc[member_index,'Website']=hostname
          df_Social.loc[member_index, 'Link'] = link.get_attribute('href')

  except Exception as e:
      print(f"Error extracting social media links: {e}")


In [21]:
def extract_member_info(member, company_index):
  global df,df_categories,df_Memmber,df_Social
  try:
      name = member.find_element(By.CSS_SELECTOR, '[data-testid="teamMemberName"]').text
  except:
      name = ' '

  try:
      member_position = member.find_element(By.CSS_SELECTOR, '[data-styleid="team-member-position"]').text
  except:
      member_position = ' '

  try:
      member_company = member.find_element(By.CSS_SELECTOR, '[data-styleid="team-member-company"]').text
  except:
      member_company = ' '

  member_index = len(df_Memmber)
  df_Memmber.loc[member_index, 'ID_Company'] = company_index
  df_Memmber.loc[member_index, 'Name'] = name
  df_Memmber.loc[member_index, 'Title Member Position'] = member_position
  df_Memmber.loc[member_index, 'Title Member Company'] = member_company


In [22]:
def extract_categories(category_elements, company_index):
  global df,df_categories,df_Memmber,df_Social
  try:
      for category in category_elements:
          cat_index = len(df_categories)
          df_categories.loc[cat_index, 'ID_Company'] = company_index
          df_categories.loc[cat_index, 'Category has to company'] = category.text
  except Exception as e:
      print(f"Error extracting categories: {e}")


In [23]:
html_code = """
<!DOCTYPE html>
<html>
<head>
  <style>
    #container {
      position: relative;
      width: 100%;
      height: 200px;
    }
    #rocket {
      position: absolute;
      top: 50%;
      transform: translateY(-50%);
      transition: left 5s linear;
    }
  </style>
</head>
<body>
  <div id="container">
    <h3>Rocket Animation</h3>
    <img id="rocket" src="https://emojicdn.elk.sh/rocket" width="100" height="100">
    <script>
      // Get the rocket element
      var rocket = document.getElementById('rocket');
      var container = document.getElementById('container');
      var containerWidth = container.offsetWidth;
      var rocketWidth = rocket.offsetWidth;

      // Animate the rocket
      var leftPosition = -rocketWidth;
      var animateRocket = function() {
        leftPosition += containerWidth;
        if (leftPosition > containerWidth) {
          leftPosition = -rocketWidth;
        }
        rocket.style.left = leftPosition + 'px';
        setTimeout(animateRocket, 5000);  // 5 seconds interval
      };
      animateRocket();
    </script>
  </div>
</body>
</html>
"""

# Display the HTML code to create the animation


In [24]:
def get_data():
  global driver,df,df_categories,df_Memmber,df_Social
  parent_elements = driver.find_elements(By.CSS_SELECTOR, 'div.css-1tjqkqx')
  for child in parent_elements:
    link = child.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
    index = len(df)
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])

    while 1:
      try:
        time.sleep(2)
        driver.get(link)
        break
      except:
        pass
    time.sleep(2)
    title = get_text('[data-styleid="profile-card-user-name"]')
    country = get_text('[data-styleid="product-subtitle1"]')
    website = get_text('[data-styleid="profile-link-site"] h6')
    exhibitor_Hall = get_text('[data-styleid="exhibitorHallBlock"] [data-styleid="exhibitorHall"]')
    exhibitor_stand = get_text('[data-styleid="exhibitorHallBlock"] [data-styleid="exhibitorStand"]')
    logo = get_text('[data-testid="profileCard"] img', 'src')
    categories = driver.find_elements(By.CSS_SELECTOR, '[data-testid="categoriesList"] > *')
    social_media_links = driver.find_elements(By.CSS_SELECTOR, '[data-styleid="social-media-block"] [data-testid="socialList"] a')
    members = driver.find_elements(By.CSS_SELECTOR, '[id="team_members"] .slick-slide > div > div > div')
    about = get_text('[data-testid="description"]')
    df.loc[index,'Title']=title
    df.loc[index,'Country']=country
    df.loc[index,'Website']=website
    df.loc[index,'Exhibitor Hall']=exhibitor_Hall
    df.loc[index,'Exhibitor Stand']=exhibitor_stand
    df.loc[index,'logo']=logo
    df.loc[index,'About']=about

    print(f'________________________________________________{index}_______________________________________________________________')

    try:
        extract_categories(categories, index)
    except Exception as e:
        print(f"Error extracting categories for company {index}: {e}")

    try:
        for member in members:
            extract_member_info(member, index)
    except Exception as e:
        print(f"Error extracting member info for company {index}: {e}")

    try:
        extract_social_media_links(social_media_links, index)
    except Exception as e:
        print(f"Error extracting social media links for company {index}: {e}")

    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    clear_output(wait=True)
    display(df)
    save_df()







In [25]:
def save_df():
  global df,df_categories,df_Memmber,df_Social
  df.to_csv('Company.csv')
  df_categories.to_csv('category.csv')
  df_Memmber.to_csv('member.csv')
  df_Memmber.to_csv('Social Media.csv')
  with pd.ExcelWriter('output.xls', engine='xlsxwriter') as writer:
      # Write each DataFrame to a separate sheet
      df.to_excel(writer, sheet_name='Comapny', index=False)
      df_categories.to_excel(writer, sheet_name='Category', index=False)
      df_Memmber.to_excel(writer, sheet_name='Memmber', index=False)
      df_Social.to_excel(writer, sheet_name='Social Media', index=False)


In [ ]:
def start_collect():
  global driver,df
  for i in range(6, 68):
    url = f'https://wineparis-vinexpo.com/newfront/marketplace/exhibitors?limit=60&pageNumber={i}'

    s="""  function scrollDownAndUp() {
        // Scroll down to the bottom
        window.scroll({
          top: document.body.scrollHeight,
          behavior: 'smooth'
        });

        // Wait for a brief moment
        setTimeout(() => {
          // Scroll up to the top
          window.scroll({
            top: 0,
            behavior: 'smooth'
          });
        }, 2000); // Adjust the duration as needed
      }

      // Call the function every 3 seconds (adjust the interval as needed)
      setInterval(scrollDownAndUp, 3000);
    """
    try:
      while True:
        driver.get(url)
        time.sleep(50)
        driver.execute_script(s)
        time.sleep(60)
        get_data()
        save_df()
        break
    except:
      pass



start_collect()

,Title,Country,Website,Exhibitor Hall,Exhibitor Stand,logo,About
0,0.0% SOBER SPIRITS,France,www.soberspirits.com/,7.1,7.1 E 070,https://di9mr54a05a64.cloudfront.net/api-vinex...,World 's First Non-Alcoholic Spirits made from...
1,3 R,France,www.gin3r.com,7.1,7.1 C 062,https://di9mr54a05a64.cloudfront.net/api-vinex...,Our Norman gin is made in the heart of the Pay...
2,6 in Piemonte,Italy,6inpiemonte.it,2.2,2.2 D 071,https://di9mr54a05a64.cloudfront.net/api-vinex...,"Barolo, Barbaresco, Gattinara, Ghemme, Monferr..."
3,Abadía de Acón - DO Ribera del Duero,Spain,www.abadiadeacon.com,5.1,5.1 E 045,https://di9mr54a05a64.cloudfront.net/api-vinex...,Abadía de Acón is a family project located in ...
4,Abbaye de Lérins,France,www.excellencedelerins.com,4,4 D 307,https://di9mr54a05a64.cloudfront.net/api-vinex...,The 21 Cistercian monks of the Abbey of Lérins...
